# Image Synthesis

### Generate Background Images

In [2]:
import os
import cv2
import random
import numpy as np

import matplotlib.pyplot as plt
import xml.etree.cElementTree as ET
from xml.dom import minidom

scan logo directory for logo images
1. define num of train images as num of logo * 100
2. define num of test images as num of logo * 40
3. get brand names from logo file names for xml file
4. get the largest height and with of logo images for largest logo dimenstion (division) caculation __shape__:returns (H,W,C)
5. create 3 size for each logo and add to logo collection (logo_name:[sm,md,lg])


to get atleast 6 regions 

1080x1920 largest logo size should be such size that there can be 6 of them on one image



In [ ]:
def logo_collection_generator(logo_dir_path='',bgr_size = (1920,1080)):

    logo_img_list = os.listdir(logo_dir_path)

    num_train_img = len(logo_img_list)*300
    num_test_img = len(logo_img_list)*100

    largest_width = int(bgr_size[0]/3)
    largest_height = int(bgr_size[1]/3)

    logo_collection = []

    for logo in logo_img_list:
        logo_name = os.path.splitext(logo)[0]
        img = cv2.imread('./logo/{}'.format(logo),-1)

        brand={}

        height = img.shape[0]
        width = img.shape[1]
        
        if height > largest_height or width > largest_width:
            reduction_ratio = largest_height/height
            height= round(reduction_ratio*height)
            width = round(reduction_ratio*width)
        '''
        big_logo = cv2.resize(img,(round(height/1.25),round(width/1.25)))
        med_logo= cv2.resize(img,(round(height/2),round(width/2)))
        #sml_logo = cv2.resize(img,(round(height/4),round(width/4)))
        
        brand['{}_big'.format(logo_name)] = big_logo
        brand['{}_med'.format(logo_name)] = med_logo
        #brand['{}_sml'.format(logo_name)] = sml_logo
        '''
        brand['{}'.format(logo_name)] = img

        logo_collection.append(brand)
        print(height,width)
    return logo_collection,largest_width,largest_height,num_train_img,num_test_img
        

In [ ]:
def region_position_generator(largest_height,largest_width,size=(1920,1080)):

    rows = int(size[1]/largest_height)
    cols = int(size[0]/largest_width)

    row_unit = int(size[1]/rows)
    col_unit = int(size[0]/cols)

    region_matrix = [] 

    for x in range(1,cols+1): #width
        for y in range(1,rows+1): #height
            region_matrix.append((x,y))

    position_matrix = []

    for x in range(0,col_unit//2): #width 
        for y in range(0,row_unit//2): # height
            position_matrix.append((x,y))
    
    return region_matrix,position_matrix,row_unit,col_unit

In [ ]:
def logo_cordinate_generator(logo_collection,region_matrix,position_matrix,row_unit,col_unit):

    logo_coordinate = []
    for brand in logo_collection:
        for logo in list(brand.keys()):
            print(logo)
            rand_num = random.randint(0,len(region_matrix)-1)
            position = position_matrix[random.randint(0,len(position_matrix)-1)]
            region = region_matrix[rand_num]

            coordinates = (position[0]+((region[0]-1)*col_unit),position[1]+((region[1]-1)*row_unit))

            xmax = brand[logo].shape[1]+coordinates[0]
            ymax = brand[logo].shape[0]+coordinates[1]
            
            position_regected = []
            while xmax > region[0]*col_unit or ymax > region[1]*row_unit:
                
                #print(f'while: {xmax} > {region[0]*col_unit} or {ymax} > {region[1]*row_unit}',region,position)
                
                position_regected.append(position)
                position_matrix = [x for x in position_matrix if x not in position_regected]
                
                rand_num = random.randint(0,len(region_matrix)-1)
                region = region_matrix[rand_num]
                position = position_matrix[random.randint(0,len(position_matrix)-1)]
                coordinates = (position[0]+((region[0]-1)*col_unit),position[1]+((region[1]-1)*row_unit))

                xmax = brand[logo].shape[1]+coordinates[0]
                ymax = brand[logo].shape[0]+coordinates[1]
            
            region_matrix.remove(region)

            logo_coordinate.append({'{}'.format(logo):(brand[logo],coordinates)})

    return logo_coordinate

In [ ]:
def xml_label_generator(directory,count,obj,shape=(1080,1920,3)):
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "train"
    ET.SubElement(root, "filename").text = "{0}.jpg".format(count)
    ET.SubElement(root, "path").text = os.getcwd()+"\\train\\{}.jpg".format(count)

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = "{}".format(str(shape[1]))
    ET.SubElement(size, "height").text = "{}".format(str(shape[0]))
    ET.SubElement(size, "depth").text = "{}".format(str(shape[2]))

    ET.SubElement(root, "segmented").text = "0"

    for obj in obj:
        object = ET.SubElement(root, "object")
        ET.SubElement(object, "name").text = obj[0]
        ET.SubElement(object, "pose").text = "Unspecified"
        ET.SubElement(object, "truncated").text = "0"
        ET.SubElement(object, "difficult").text = "0"

        bndbox = ET.SubElement(object, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(obj[1][0])
        ET.SubElement(bndbox, "ymin").text = str(obj[1][1])
        ET.SubElement(bndbox, "xmax").text = str(obj[1][2])
        ET.SubElement(bndbox, "ymax").text = str(obj[1][3])
        print(obj)
    xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent='\t')
    xmlstr = xmlstr.replace('<?xml version="1.0" ?>\n','')
    with open("./{0}/{1}.xml".format(directory,count), "w") as f:
        f.write(xmlstr)

In [ ]:
def data_generator(logo_coordinate,bgr_img,directory='',count=0,):
    obj = []

    for logo in logo_coordinate:
        
        confirm = random.randint(0,2)
        
        if confirm == 0:
            continue
        
        #logo_count += 1
        logo_name = list(logo.keys())[0]
        
        logo_img = logo[logo_name][0]
        coordinates = logo[logo_name][1]
        logo_size = logo_img.shape
        
        x_offset,y_offset = coordinates[0],coordinates[1]
        
        y1, y2 = y_offset, y_offset + logo_size[0]
        x1, x2 = x_offset, x_offset + logo_size[1]
        
        alpha_logo = logo_img[:, :, 3] / 255.0
        alpha_bgr = 1.0 - alpha_logo
        
        for c in range(0, 3):
            bgr_img[y1:y2, x1:x2, c] = (alpha_logo * logo_img[:, :, c] +
                                    alpha_bgr * bgr_img[y1:y2, x1:x2, c])
        
        random_add = []
        for x in range(0,4):
            random_add.append(random.randint(0,8))

        obj.append((str(logo_name).split('_')[0],[x1-random_add[0],y1-random_add[2],x2+random_add[3],y2+random_add[1]]))
        
        #bgr_img = noisy(noise[random.randint(0,3)],bgr_img)
        #bgr_img = cv2.resize(bgr_img,(480,360))
            
        #bgr_img = cv2.rectangle(bgr,(x1-random_add[0],y1-random_add[2]),(x2+random_add[3],y2+random_add[1]),(0,255,0),2)
        #cv2.imshow("final", bgr_img)
        #cv2.waitKey(0)
        
        cv2.imwrite('./{0}/{1}.jpg'.format(directory,count),bgr_img)
        print('Writing Generated Image: Done')
        shape = bgr_img.shape
        xml_label_generator(directory,count,obj,shape)
    cv2.destroyAllWindows()
    #return logo_count

In [ ]:
file_list = os.listdir('./background/')

size = (640,480)

logo_collection,largest_width,largest_height,num_train_img,num_test_img = logo_collection_generator('./logo',bgr_size=size)
print('Logo Collection: Done')
print(largest_width,largest_height,num_train_img,num_test_img)
dataset = ['train','test']

for data in dataset:
    if data == 'train':
        count = num_train_img
        print('Train Data Generation: Started')
    else:
        count = num_test_img
        print('Test Data Generation: Started')
    for i in range(1,count+1):
        file = file_list[random.randint(0,99)]
        
        print('Read Background Image: ',file)
        bgr = cv2.imread('./background/{}'.format(file))
        bgr = bgr[200:200+size[1], 600:600+size[0]]
        
        region_matrix,position_matrix,row_unit,col_unit = region_position_generator(largest_height,largest_width,size)
        print('Generating Region, position matrix: Done')

        logo_coordinate = logo_cordinate_generator(logo_collection,region_matrix,position_matrix,row_unit,col_unit)
        print('Generating Logo Coordinates: Done')
        
        data_generator(logo_coordinate,bgr,directory=data,count=i)
        

In [5]:
xml_files = [file for file in os.listdir('./test') if file.endswith('.xml')]
classes = []
class_dict = {'kent':0, 'ipl':0}
for files in xml_files:
    content = ET.parse('./test'+'/{0}'.format(files))
    root = content.getroot()
    for child in root:
        if child.tag == 'object':
            classes.append(child.find('name').text)

for element in classes:
    if element == 'ipl':
        class_dict['ipl'] += 1
    elif element == 'kent':
        class_dict['kent'] += 1
print(class_dict)

{'kent': 397, 'ipl': 205}
